# Initial codes

This notebook illustrates how Bayesian inference can be used to infer response rates of each group (basket) in a Basket trial.

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
from os.path import exists

sys.path.append('..')
sys.path.append('.')

In [3]:
import numpy as np
import pandas as pd
import arviz as az
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [4]:
from pyBasket.common import load_obj
from pyBasket.common import (
    GROUP_STATUS_COMPLETED_EFFECTIVE,
    MODEL_INDEPENDENT_BERN,
    MODEL_BHM,
    MODEL_PYBASKET
)

In [5]:
def get_output_filenames(result_dir, scenario, with_clustering_info, n_clusters):
    base_filename = f'scenario_{scenario}_clustering_{with_clustering_info}'
    if with_clustering_info:
        base_filename += f'_ncluster_{n_clusters}'

    out_trial_results = os.path.join(result_dir, base_filename + '_trial_results.p')
    return out_trial_results

In [6]:
out_dir = os.path.abspath(os.path.join('..', 'scripts', 'results'))
out_dir

'/Users/joewandy/Work/git/pyBasket/scripts/results'

In [8]:
scenarios = range(0, 7)
clustering = [False, True]
n_clusters = [5, 10]

In [9]:
def gather_data(scenarios, clustering, n_clusters, out_dir):
    data = {}

    for scenario in scenarios:
        for cl in clustering:
            nc_values = [None] if not cl else n_clusters

            for nc in nc_values:
                try:
                    fname = get_output_filenames(out_dir, scenario, cl, nc)
                    print(fname)                
                    trial_results = load_obj(fname)
                    key = (scenario, cl, nc)
                    data[key] = trial_results
                except FileNotFoundError:
                    pass
                
    return data

In [10]:
data = gather_data(scenarios, clustering, n_clusters, out_dir)

/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_3_clustering_False_trial_results

In [11]:
def get_analysis(data, trial_idx, scenario, use_clustering, num_clusters, last_only=True):
    key = (scenario, use_clustering, num_clusters)
    analysis_results = {}

    if key not in data:
        print(f"No trial result found for key {key}")
        return

    trial_result = data[key]

    if trial_idx >= len(trial_result):
        print(f"Trial index {trial_idx} out of range.")
        return

    for analysis_name in trial_result[trial_idx].idfs:
        idfs = trial_result[trial_idx].idfs[analysis_name]
        if last_only and idfs:
            idfs = idfs[-1]
        analysis_results[analysis_name] = idfs

    return analysis_results

def get_analysis_by_name(analysis_results, analysis_name):
    if analysis_name not in analysis_results:
        print(f"No analysis results found for {analysis_name}")
        return

    idfs = analysis_results[analysis_name]

    return idfs


def display_analysis(idfs):
    if isinstance(idfs, list):
        for df in idfs:
            display(df)
    elif idfs is not None:
        display(idfs)
    else:
        print("No data to display.")


In [21]:
def get_true_arr(scenario):

    if scenario == 0:
        y_true = np.array([False, False, False, False, False, False])
    elif scenario == 1:
        y_true = np.array([True, False, False, False, False, False])
    elif scenario == 2:
        y_true = np.array([True, True, False, False, False, False])
    elif scenario == 3:
        y_true = np.array([True, True, True, False, False, False])
    elif scenario == 4:
        y_true = np.array([True, True, True, True, False, False])
    elif scenario == 5:
        y_true = np.array([True, True, True, True, True, False])
    elif scenario == 6:
        y_true = np.array([True, True, True, True, True, True])
        
    y_true_all = []
    for trial_idx in range(total):
        y_true_all.append(y_true)
    y_true_all = np.array(y_true_all)
    return y_true_all

In [22]:
def get_pred_arr(data, total, analysis_name, scenario, use_clustering, num_clusters):
    y_pred_all = []
    for trial_idx in range(total):

        # get the data first
        analysis_results = get_analysis(data, trial_idx, scenario, use_clustering, num_clusters)

        df = get_analysis_by_name(analysis_results, analysis_name)
        df_status = df['group_status'] == GROUP_STATUS_COMPLETED_EFFECTIVE
        y_pred = df_status.values
        y_pred_all.append(y_pred)

    y_pred_all = np.array(y_pred_all)
    return y_pred_all

|                   | Predicted Negative | Predicted Positive |
|-------------------|--------------------|--------------------|
| Actual Negative   |        TN          |        FP          |
| Actual Positive   |        FN          |        TP          |

In [23]:
def get_percent_reject(y_true_all, y_pred_all, basket_idx):

    y_true = y_true_all[:, basket_idx]
    y_pred = y_pred_all[:, basket_idx]
    
    # Calculate the basket-wise type I error rate. 
    # This corresponds to the proportion of false positives, or 
    # the cases when the null hypothesis is true (i.e., the treatment 
    # is not effective), but it's incorrectly rejected 
    # (leading us to wrongly believe that the treatment is effective).
    
    count = np.sum((y_true == False) & (y_pred == True))
    total = y_true_all.shape[0]
    final = f'I:{count / total * 100:.2f}'
    # final = count / total * 100
        
    if count == 0: # 
        
        # If we can't calculate the type I error, then report the power instead
        # The basket-wise power corresponds to the proportion of true positives, 
        # or the cases when the null hypothesis is false (i.e., the treatment is 
        # effective), and it's correctly rejected (leading us to correctly conclude 
        # that the treatment is effective).
        
        count = np.sum((y_true == True) & (y_pred == True))    
        total = y_true_all.shape[0]
        final = f'p:{count / total * 100:.2f}'
        # final = count / total * 100
        
    return final

In [24]:
def get_metrics(y_true_all, y_pred_all, basket_idx):
    cm = confusion_matrix(y_true_all[:, basket_idx], y_pred_all[:, basket_idx], labels=[False, True])
    TN = float(cm[0][0])
    FP = float(cm[0][1])
    FN = float(cm[1][0])
    TP = float(cm[1][1])
    accuracy = (TN + TP) / (TN + FP + FN + TP)

    try:
        precision = TP / (TP + FP)
    except ZeroDivisionError:
        precision = 0.0

    try:
        recall = TP / (TP + FN)
    except ZeroDivisionError:
        recall = 0.0

    try:
        sensitivity = TN / (TN + FP)
    except ZeroDivisionError:
        sensitivity = 0.0

    try:
        f1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0.0
        
    try:
        fpr = FP / (FP + TN)
    except ZeroDivisionError:
        fpr = 0.0
    
    return TN, FP, FN, TP, accuracy, precision, recall, sensitivity, f1, fpr

In [25]:
def analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters):
    df_data = []
    for analysis_name in analysis_names:
        for scenario_idx in range(n_scenario):
            y_true_all = get_true_arr(scenario_idx)
            y_pred_all = get_pred_arr(data, total, analysis_name, scenario_idx, use_clustering, num_clusters)

            for basket_idx in range(n_baskets):
                reject = get_percent_reject(y_true_all, y_pred_all, basket_idx)
                # TN, FP, FN, TP, accuracy, precision, recall, sensitivity, f1, fpr = get_metrics(y_true_all, y_pred_all, basket_idx)
                row = [scenario_idx, analysis_name, basket_idx, reject]
                df_data.append(row)

    reject_df = pd.DataFrame(df_data, columns=['scenario', 'analysis_name', 'basket_idx', 'reject'])
    reshaped_df = reject_df.pivot(index=['scenario', 'analysis_name'], columns='basket_idx', values='reject')
    return reshaped_df


In [27]:
total = 500
n_scenario = 7
n_baskets = 6
analysis_names = [
    MODEL_INDEPENDENT_BERN,
    MODEL_BHM,
    MODEL_PYBASKET
]

In [28]:
use_clustering = False
num_clusters = None
df = analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters)
df

basket_idx                       0         1        2         3         4  \
scenario analysis_name                                                      
0        BHM               I:10.40   I:11.80  I:11.40   I:10.60   I:13.40   
         independent_bern   I:7.80   I:11.00  I:10.80    I:9.80   I:14.00   
         pyBasket           I:7.60   I:10.40   I:8.20    I:8.20   I:14.60   
1        BHM               p:58.20   I:33.20  I:33.60   I:32.60   I:32.80   
         independent_bern  p:81.00    I:8.60  I:11.20   I:12.60   I:10.40   
         pyBasket          p:77.60    I:6.00   I:4.80    I:6.80    I:4.80   
2        BHM               p:81.80   p:84.20  I:48.80   I:50.20   I:49.00   
         independent_bern  p:82.40   p:83.60  I:12.60    I:9.60    I:9.40   
         pyBasket          p:62.60   p:65.00   I:3.40    I:2.20    I:2.40   
3        BHM               p:93.60   p:92.60  p:92.60   I:62.40   I:63.80   
         independent_bern  p:80.60   p:81.20  p:83.20    I:7.20   I:12.00   
         pyBasket          p:55.00   p:49.00  p:50.40    I:1.40    I:0.80   
4        BHM               p:97.40   p:98.00  p:97.80   p:97.40   I:76.00   
         independent_bern  p:83.60   p:83.40  p:82.20   p:85.40    I:9.80   
         pyBasket          p:40.40   p:39.80  p:39.40   p:38.60    I:0.40   
5        BHM               p:99.00   p:98.80  p:99.40   p:98.40   p:98.80   
         independent_bern  p:82.60   p:82.00  p:82.60   p:78.80   p:84.80   
         pyBasket          p:24.20   p:27.00  p:26.80   p:27.00   p:25.80   
6        BHM               p:99.80  p:100.00  p:99.80  p:100.00  p:100.00   
         independent_bern  p:79.60   p:83.00  p:82.00   p:81.00   p:83.40   
         pyBasket          p:16.20   p:17.80  p:18.20   p:17.00   p:17.40   

basket_idx                       5  
scenario analysis_name              
0        BHM               I:11.40  
         independent_bern   I:9.60  
         pyBasket           I:9.20  
1        BHM               I:33.60  
         independent_bern  I:10.20  
         pyBasket           I:6.20  
2        BHM               I:53.20  
         independent_bern   I:9.00  
         pyBasket           I:2.20  
3        BHM               I:64.20  
         independent_bern  I:10.40  
         pyBasket           I:1.60  
4        BHM               I:74.60  
         independent_bern  I:10.00  
         pyBasket           I:0.20  
5        BHM               I:89.00  
         independent_bern   I:9.80  
         pyBasket           I:0.20  
6        BHM               p:99.80  
         independent_bern  p:82.40  
         pyBasket          p:15.40

In [29]:
use_clustering = True
num_clusters = 5
df = analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters)
df

basket_idx                       0        1        2        3        4  \
scenario analysis_name                                                   
0        BHM                I:0.20   I:0.20   I:0.20   I:0.80   I:1.40   
         independent_bern   I:1.60   I:2.00   I:2.60   I:2.60   I:5.80   
         pyBasket           I:3.40   I:3.60   I:4.00   I:5.20   I:6.60   
1        BHM               p:35.20  I:10.60   I:9.40  I:11.20   I:8.80   
         independent_bern  p:75.00  I:13.00  I:11.40  I:14.60  I:11.40   
         pyBasket          p:70.00  I:10.20   I:9.20   I:9.00   I:7.20   
2        BHM                p:6.20   p:3.80   I:0.20   I:0.40   I:0.20   
         independent_bern  p:18.80  p:17.00   I:0.20   I:1.40   I:1.00   
         pyBasket          p:20.20  p:14.60   I:0.60   I:0.80   I:1.00   
3        BHM               p:16.60  p:18.80  p:16.40   I:4.20   I:4.60   
         independent_bern  p:41.20  p:41.20  p:40.20   I:2.60   I:2.80   
         pyBasket          p:31.60  p:31.20  p:35.20   I:2.00   I:2.20   
4        BHM               p:48.40  p:41.00  p:50.00  p:45.00  I:35.20   
         independent_bern  p:61.20  p:41.40  p:65.20  p:50.00   I:7.00   
         pyBasket          p:43.20  p:20.00  p:44.80  p:28.20   I:2.20   
5        BHM               p:10.20  p:12.40  p:13.40  p:15.60  p:14.40   
         independent_bern  p:18.80  p:19.80  p:23.40  p:26.00  p:34.80   
         pyBasket           p:9.20  p:13.20  p:15.20  p:16.60  p:25.00   
6        BHM                p:4.00   p:4.00   p:6.00   p:3.00   p:5.00   
         independent_bern   p:9.00  p:10.60  p:17.40   p:7.20  p:15.60   
         pyBasket           p:4.40   p:7.00   p:8.20   p:4.40   p:8.40   

basket_idx                       5  
scenario analysis_name              
0        BHM                I:0.60  
         independent_bern   I:3.80  
         pyBasket           I:4.40  
1        BHM                I:9.20  
         independent_bern  I:10.60  
         pyBasket           I:6.60  
2        BHM                I:0.20  
         independent_bern   I:1.80  
         pyBasket           I:0.80  
3        BHM                I:6.20  
         independent_bern   I:7.60  
         pyBasket           I:4.80  
4        BHM               I:34.80  
         independent_bern  I:12.20  
         pyBasket           I:3.60  
5        BHM                I:7.80  
         independent_bern   I:3.40  
         pyBasket           I:1.40  
6        BHM                p:4.60  
         independent_bern  p:16.80  
         pyBasket           p:7.00

In [30]:
use_clustering = True
num_clusters = 10
df = analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters)
df

basket_idx                       0        1        2        3        4  \
scenario analysis_name                                                   
0        BHM                I:2.20   I:2.80   I:3.00   I:2.60   I:3.60   
         independent_bern  I:28.60  I:28.80  I:22.60  I:24.20  I:26.00   
         pyBasket          I:26.20  I:27.80  I:21.60  I:22.60  I:23.00   
1        BHM               p:26.20   I:9.00  I:10.60  I:11.00   I:9.20   
         independent_bern  p:84.00  I:33.20  I:40.00  I:34.00  I:34.00   
         pyBasket          p:81.60  I:24.00  I:32.00  I:29.80  I:28.20   
2        BHM               p:43.20  p:40.00  I:23.60  I:23.40  I:22.80   
         independent_bern  p:88.00  p:84.60  I:36.00  I:37.00  I:36.40   
         pyBasket          p:84.60  p:79.80  I:23.60  I:25.20  I:26.00   
3        BHM               p:25.40  p:26.80  p:22.80  I:10.00   I:9.80   
         independent_bern  p:72.20  p:71.80  p:71.80  I:19.60  I:19.60   
         pyBasket          p:65.20  p:66.40  p:67.80  I:11.00  I:13.80   
4        BHM               p:13.40  p:15.20  p:13.40  p:15.00   I:5.80   
         independent_bern  p:56.00  p:58.20  p:59.00  p:65.80  I:13.80   
         pyBasket          p:47.40  p:50.80  p:50.00  p:55.40   I:7.60   
5        BHM               p:25.40  p:25.20  p:26.20  p:24.40  p:26.40   
         independent_bern  p:64.60  p:62.00  p:62.80  p:61.40  p:63.60   
         pyBasket          p:51.60  p:45.20  p:52.20  p:47.80  p:49.00   
6        BHM               p:12.00  p:11.40  p:11.20  p:10.80  p:13.00   
         independent_bern  p:49.20  p:41.20  p:43.20  p:45.40  p:47.60   
         pyBasket          p:35.80  p:32.00  p:28.80  p:34.20  p:36.00   

basket_idx                       5  
scenario analysis_name              
0        BHM                I:2.40  
         independent_bern  I:25.00  
         pyBasket          I:20.60  
1        BHM               I:10.60  
         independent_bern  I:30.80  
         pyBasket          I:24.40  
2        BHM               I:22.20  
         independent_bern  I:35.00  
         pyBasket          I:26.00  
3        BHM                I:9.60  
         independent_bern  I:22.20  
         pyBasket          I:14.20  
4        BHM                I:5.20  
         independent_bern  I:13.80  
         pyBasket           I:8.60  
5        BHM               I:17.00  
         independent_bern  I:16.00  
         pyBasket           I:7.20  
6        BHM               p:12.40  
         independent_bern  p:50.40  
         pyBasket          p:39.60